In [7]:
import os
import numpy as np
import pandas as pd 

from stable_baselines3 import PPO, SAC, TD3, DDPG, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from stable_baselines3.common.callbacks import CheckpointCallback, CallbackList

import sys

from torch.utils.data.dataset import Dataset, random_split

import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import gym

path = r"c:\Users\spige\memoire\gym-donkeycar-retry\gym-donkeycar\CarConsumptionModel"
sys.path.insert(0, path)

from donkey_environment.ConsumptionWrapper import ConsumptionWrapper
from utils.callbacks import CustomProgressBarCallback, SaveObservations
from utils.ExpertDataset import ExpertDataSet


In [13]:
expert_dataset = ExpertDataSet()
expert_dataset.load_dataset(file_path="../data/rollout/dataset_clamped.npz")

print(f" observation shape : {expert_dataset.observations.shape} \n")
print(f" action shape : {expert_dataset.actions.shape} \n")
    



 observation shape : (193563, 120, 160, 3) 

 action shape : (193563, 2) 



In [11]:
def transform_observation(obs: np.ndarray) -> np.ndarray:
    # transform (1332, 120, 160, 3) to (1332, 3, 120, 160)
    transformed_obs = np.transpose(obs, (0, 3, 1, 2))
    return transformed_obs

def clamp(x: float, min_value: float, max_value: float) -> float:
    return max(min(x, max_value), min_value)

In [12]:
print(expert_dataset.actions.shape)

clamped_actions = np.ndarray(shape=expert_dataset.actions.shape, dtype=np.float32)
for i, action in enumerate(expert_dataset.actions):

    steering = action[0]
    throttle = action[1]

    clamped_steering = clamp(steering, -1.0, 1.0)
    clamped_throttle = clamp(throttle, -1.0, 1.0)

    clamped_actions[i] = np.array([clamped_steering, clamped_throttle])

# store the expert dataset in a new file
np.savez("../data/rollout/dataset_clamped.npz", obs=expert_dataset.observations, action=clamped_actions)

(193563, 2)
